In [1]:
### import numpy as np
from collections import defaultdict
import pandas as pd
from tqdm.notebook import tqdm
import glob
import numpy as np
import multiprocessing
import os
import pickle

import glob
from collections import Counter

DEBUG=False
SAMPLING = 1  # Reduce it to improve performance

In [3]:
import gc

In [ ]:
import sys
def gen_pairs(df):
    df = df.query('session % @SAMPLING == 0').groupby('session', as_index=False, sort=False).apply(lambda g: g.tail(30)).reset_index(drop=True)
    df = pd.merge(df, df, on='session')
    pairs = df.query('abs(ts_x - ts_y) < 24 * 60 * 60 * 1000 and aid_x != aid_y')[['session', 'aid_x', 'aid_y', 'ts_x', 'type_y']]\
        .drop_duplicates(['session', 'aid_x', 'aid_y'])
    return pairs[['aid_x', 'aid_y', 'ts_x', 'type_y']].values


def gen_aid_pairs():
    all_pairs = defaultdict(lambda: Counter())
    with tqdm(glob.glob('../../../../Downloads/otto-chunk-data-inparquet-format/*_parquet/*'), desc='Chunks') as prog:
        with multiprocessing.Pool(4) as p:
            for idx, chunk_file in enumerate(prog):
                chunk = pd.read_parquet(chunk_file)#.drop(columns=['type'])
                pair_chunks = p.map(gen_pairs, np.array_split(chunk.head(100000000 if not DEBUG else 10000), 120))
                for pairs in pair_chunks:
                    for aid1, aid2, ts, typ in pairs:
                        w = 1 + 3*(ts - 1659304800025)/(1662328791563-1659304800025)
                        # HERE WE CAN BOOST WEIGHT, i.e. IF TYP=="ORDERS": W *= 10.0
                        # THEN SAVE THIS MATRIX AS THE "ORDERS" MATRIX
                        # WE CAN MAKE 3 DIFFERENT CO-VISITATION MATRICES
                        all_pairs[aid1][aid2] +=w
                prog.set_description(f'Mem: {sys.getsizeof(object) // (2 ** 20)}MB')

                if DEBUG and idx >= 2:
                    break
                del chunk, pair_chunks
                gc.collect()
    return all_pairs
#
# if os.path.exists(TOP_20_CACHE):
#     print('Reading top20 AIDs from cache')
#     top_20 = pickle.load(open(TOP_20_CACHE, 'rb'))
# else:
all_pairs = gen_aid_pairs()
df_top_20 = []
for aid, cnt in tqdm(all_pairs.items()):
    df_top_20.append({'aid1': aid, 'aid2': [aid2 for aid2, freq in cnt.most_common(20)]})

df_top_20 = pd.DataFrame(df_top_20).set_index('aid1')
top_20 = df_top_20.aid2.to_dict()
import pickle
with open('top_20_aids.pkl', 'wb') as f:
    pickle.dump(top_20, f)


Chunks:   0%|          | 0/146 [00:00<?, ?it/s]